In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Agentspace custom agent with Vertex AI session

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/search/agentspace/agentspace_session.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fsearch%2Fagentspace%2Fagentspace_session.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/search/agentspace/agentspace_session.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/search/agentspace/agentspace_session.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/search/agentspace/agentspace_session.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/search/agentspace/agentspace_session.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/search/agentspace/agentspace_session.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/search/agentspace/agentspace_session.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/search/agentspace/agentspace_session.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>

| Authors |
| --- |
| Rupjit Chakraborty|
| Sharmila Devi |
| Parag Mhatre |

## Overview : Custom agent with sessions in Agentspace

This notebook demonstrates an end-to-end example of a custom agent integrated with Agentspace that uses [VertexAISessionService](https://google.github.io/adk-docs/sessions/session/#managing-sessions-with-a-sessionservice). In this case, a simple example of a travel planner has been demonstrated, which takes the help of two sub-agents, viz. query completeness checker and itinerary 
generator. This agent helps a user plan their travel based on their travel destination and time of travel. Due to the use of Vertex AI Session service, this agent can also modify generated itineraries.

### Background

Custom agents integrated with Agentspace may need to leverage a user's previous conversations and preferences to answer a query satisfactorily. By default, [`AdkApp` template is already connected to Vertex AI Agent Engine Sessions through `session_service`](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/sessions/manage-sessions-adk#python). However, this session service is not persistent. There are two persistent session service - `VertexAiSessionService` and `DatabaseSessionService`. The use of `VertexAiSessionService` has been demonstrated below.

### Business Application Scenario

We demonstrate a simple scenario where a user is planning a travel to a certain location. The user provides destination information first but to generate a suitable itinerary the agent also needs a time information. The agent requests the user for time information and after the user provides it, the agent generates the itinerary. After the itinerary generation the user requests modification based on a certain criteria and agent updates the itinerary based on that.

### Notebook Overview

This notebook guides you through the following key steps:

1. Creation of custom travel agent
    a. Create the sub-agents - query completeness checker and itinerary generator  
    b. Create the root agent to call the sub-agents to respond to a user query
2. Local demo of the custom travel agent using VertexAISessionService
    a. Create an Agent Engine instance and VertexAISessionService  
    b. Demo user query response using the VertexAISessionService
3. Update/Deploy the custom travel agent to agent engine and demo user queries 
    a. Update the agent engine instance created in 2.  
    b. Demo user query response using the VertexAISessionService via Agent Engine remote agent.
4. Integrate the agent with Agentspace
    a. Register the agent in Agentspace

In [ ]:
%pip install --upgrade google-adk google-genai google-cloud-aiplatform

## Imports

In [ ]:
from dotenv import load_dotenv
from google import adk
from google.adk.agents import Agent
from google.adk.sessions import VertexAiSessionService
from google.adk.tools.agent_tool import AgentTool
from google.genai import types
import vertexai
from vertexai import agent_engines
from vertexai.preview import reasoning_engines

## Constants and Initializations

In [ ]:
# TODO for Developer: Update project, location, bucket details

PROJECT_ID = "[your-project-name]"
PROJECT_NUMBER = "[your-project-number]"
LOCATION = "[your-location]"
STAGING_BUCKET = "[your-bucket]"

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

## Set and load environment variables

In [ ]:
# environment variables

ENV_SETTINGS = """
# Choose Model Backend: 0 -> API Key, 1 -> Vertex AI
GOOGLE_GENAI_USE_VERTEXAI=1

# --- API Key Configuration (if GOOGLE_GENAI_USE_VERTEXAI=0) ---
# Obtain your API key from Google AI Studio or Google Cloud console
GOOGLE_API_KEY=YOUR_GOOGLE_API_KEY

# --- Vertex AI Backend Configuration (if GOOGLE_GENAI_USE_VERTEXAI=1) ---
# Your Google Cloud Project ID
GOOGLE_CLOUD_PROJECT=[your-project-name]
# The location (region) for Vertex AI services
GOOGLE_CLOUD_LOCATION=[your-location]
"""

In [ ]:
# Save and load environment variables

with open(".env", "w") as f_handle:
    f_handle.write(ENV_SETTINGS)

load_dotenv()

## Create agent and sub-agents

In [ ]:
# prompt to check if the user query is complete in

QUERY_CHECK_PROMPT = """
You are an expert travel planner and can check a user query for completeness and correctness

INSTRUCTIONS
Given the user query check if 
1. the query is related to travel itinerary planning
2. the query mandatorily contains both place and time of travel

EXAMPLE
travel is Aug --> incomplete query as no place of travel information
travel to Karnataka --> incomplete query as no time of travel information
travel to Karnataka in Aug --> correct query

OUTPUT
If the user query is complete: just say "query is complete"
If the user query is not complete: question user to complete the query like "Can you provide the month information ?"
"""

In [ ]:
travel_query_validator = Agent(
    model="gemini-2.5-flash",
    name="travel_query_validator",
    description="This agent checks if the user query appropriate",
    instruction=QUERY_CHECK_PROMPT,
    disallow_transfer_to_parent=False,
    disallow_transfer_to_peers=True,
)

In [ ]:
ITINERARY_PROMPT = """
You are an expert travel planner and can provide a user itinerary

INSTRUCTION
Given the user query, provide a suitable itinerary
"""

In [ ]:
planner_agent = Agent(
    model="gemini-2.5-flash",
    name="planner_agent",
    description="This agent provides a travel plan to user based on the query",
    instruction=ITINERARY_PROMPT,
    disallow_transfer_to_parent=True,
    disallow_transfer_to_peers=True,
)

In [ ]:
root_agent = Agent(
    name="root",
    model="gemini-2.5-flash",
    description=(
        "A travel agent that takes a user query and prepares a detailed itinerary with the help of sub-agents"
    ),
    generate_content_config=types.GenerateContentConfig(temperature=0.01),
    tools=[AgentTool(agent=travel_query_validator), AgentTool(agent=planner_agent)],
)

## Test the agent locally

In [ ]:
# TODO for Developer: Update user_id

import nest_asyncio

nest_asyncio.apply()

import asyncio

# Create an agent engine instance
agent_engine = agent_engines.create()
app_name = agent_engine.name.split("/")[-1]
user_id = "[session-user-id]"

# Create the ADK runner with VertexAiSessionService
session_service = VertexAiSessionService(
    project=PROJECT_ID, location=LOCATION, agent_engine_id=app_name
)
runner = adk.Runner(
    agent=root_agent, app_name=app_name, session_service=session_service
)


# Helper method to send query to the runner
async def call_agent_async(query, session_id, user_id):
    content = types.Content(role="user", parts=[types.Part(text=query)])
    events = runner.run(user_id=user_id, session_id=session_id, new_message=content)
    for event in events:
        if event.is_final_response():
            final_response = event.content.parts[0].text
            print("Agent: ", final_response)


async def main():
    session = await session_service.create_session(app_name=app_name, user_id=user_id)
    print(f"Session created with ID: {session.id}")
    user_queries = [
        "trip to London",
        "in Jan or Feb",
        "suitable for aged people",
    ]
    for user_query in user_queries:
        print(f"User Query: {user_query}")
        await call_agent_async(user_query, session.id, user_id)


if __name__ == "__main__":
    asyncio.run(main())

## Update the agent in agent engine

In [ ]:
# TODO for Developer: Update display_name (of agent)

# create a callable session service builder that can use with AdkApp


def session_service_builder():
    return VertexAiSessionService(
        project=PROJECT_ID, location=LOCATION, agent_engine_id=app_name
    )


# create the AdkApp with the session service
adk_app = reasoning_engines.AdkApp(
    agent=root_agent,
    enable_tracing=True,
    session_service_builder=session_service_builder,
)

# update the agent engine instance (created during local testing)
remote_app = agent_engines.update(
    display_name="[agent-display-name]",
    resource_name=agent_engine.name,
    agent_engine=adk_app,
    requirements=[
        "google-adk (==1.5.0)",
    ],
)

## Test the agent in agent engine

In [ ]:
# TODO for Developer: Update user_id

user_queries = ["trip to London", "in Jan or Feb", "suitable for aged people"]
user_id = "[session-user-id]"

remote_session = remote_app.create_session(user_id=user_id)

for user_query in user_queries:
    print(f"User Query: {user_query}")
    contents = types.Content(role="user", parts=[types.Part.from_text(text=user_query)])
    for event in remote_app.stream_query(
        user_id=user_id, session_id=remote_session["id"], message=contents.model_dump()
    ):
        if "text" in event["content"]["parts"][0]:
            print(f"Agent: {event['content']['parts'][0]['text']}")

## Register agent in Agentspace

In [ ]:
%%bash

curl -X POST \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
-H "X-Goog-User-Project: [your-project-id]" \
"https://discoveryengine.googleapis.com/v1alpha/projects/[your-project-id]/locations/global/authorizations?authorizationId=travel" \
-d '{
"name": "projects/[your-project-id]/locations/global/authorizations/travel",
"serverSideOauth2": {
"clientId": "[UPDATE-CLIENT-ID]",
"clientSecret": "[UPDATE-CLIENT-SECRET]",
"authorizationUri": "https://accounts.google.com/o/oauth2/v2/auth?client_id=[UPDATE-CLIENT-ID]&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform&include_granted_scopes=true&response_type=code&access_type=offline&prompt=consent",
"tokenUri": "https://oauth2.googleapis.com/token"
}
}'

In [ ]:
%%bash

curl -X POST \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
-H "X-Goog-User-Project: [your-project-id]" \
"https://discoveryengine.googleapis.com/v1alpha/projects/[your-project-id]/locations/global/collections/default_collection/engines/[your-agentspace-app-id]/assistants/default_assistant/agents" \
-d '{
"displayName": "DDL SQL Generator",
"description": "SQL generation agent to help user with SQL queries for a give schema.",
"adk_agent_definition": {
"tool_settings": {
"tool_description": "You are an expert SQL developer capable of understanding user queries and converting them to SQL"
},
"provisioned_reasoning_engine": {
"reasoning_engine": "projects/[your-project-number]/locations/us-central1/reasoningEngines/[reasoning-engine-id]"
},
"authorizations": ["projects/[your-project-number]/locations/global/authorizations/travel"]
}
}'

## Travel Agent in Agentspace

![DDL based SQL query generator agent](https://services.google.com/fh/files/blogs/ss_travel_agent.png)